In [1]:
!pip install pyspark

# intro

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Lab 1").getOrCreate()
spark

In [4]:
print(f"Spark Version: {spark.version}")

Spark Version: 3.5.3


In [5]:
spark.stop()

หลังจากนี้ ตัวแปร spark ก็จะใช้ไม่ได้แล้ว ต้องเปิดใหม่ด้วย

> spark = SparkSession.builder.appName("Lab 1").getOrCreate()



In [12]:
data = [1, 2, 3, 4, 5]

rdd = spark.sparkContext.parallelize(data)
rdd

ParallelCollectionRDD[2] at readRDDFromFile at PythonRDD.scala:289

In [15]:
rdd.getNumPartitions()

2

In [13]:
rdd_multi = rdd.map(lambda x: x* 2)
rdd_multi

PythonRDD[3] at RDD at PythonRDD.scala:53

In [14]:
rdd_multi.collect()

[2, 4, 6, 8, 10]

.collect() เอา data ที่ถูก transform มา

In [17]:
data_2 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [20]:
rdd_2 = spark.sparkContext.parallelize(data_2, 3)
rdd_2

ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:289

In [21]:
rdd.getNumPartitions()

2

In [22]:
partitions = rdd_2.glom()
partitions

PythonRDD[6] at RDD at PythonRDD.scala:53

In [23]:
partitions.collect()

[[1, 2, 3], [4, 5, 6], [7, 8, 9, 10]]

In [25]:
print(partitions.take(1))
print(partitions.take(2))

[[1, 2, 3]]
[[1, 2, 3], [4, 5, 6]]


In [31]:
rdd_2.map(lambda x: x*2)

PythonRDD[21] at RDD at PythonRDD.scala:53

In [28]:
rdd_2.map(lambda x: x*2).take(8)

[2, 4, 6, 8, 10, 12, 14, 16]

In [29]:
rdd_2.take(8)

[1, 2, 3, 4, 5, 6, 7, 8]

ข้อมูลไม่สามารถ edit เพราะมีความเป็น immutable

# คำนวณผลรวม

In [32]:
rdd_2.reduce(lambda a, b: a + b) # จับทุกตัว บวก กัน

55

In [33]:
rdd_2.reduce(lambda a, b: a * b) # จับทุกตัว คูณ กัน

3628800

In [34]:
partitions.reduce(lambda a, b: a + b) # list + list + list = list ก้อนใหญ่ ที่ไม่แบ่ง partitions

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# read files

In [45]:
with open("/content/example_data.txt", "w") as f:
  f.write("Data Science 3rd Years \n")
  f.write("Merry X Mas \n")
  f.write("Happy New Year 2025")

In [46]:
!ls

example_data.txt  sample_data


In [47]:
rdd_file = spark.sparkContext.textFile("example_data.txt")
rdd_file.collect()

['Data Science 3rd Years ', 'Merry X Mas ', 'Happy New Year 2025']

# length words each line in files

In [48]:
lengths_data = rdd_file.map(lambda line: len(line))   # count all
lengths_data.collect()

[23, 12, 19]

In [49]:
lengths_data = rdd_file.map(lambda line: len(line.replace(" ","")))   # count all alpabet
lengths_data.collect()

[19, 9, 16]

# total length words in files

In [50]:
total_length = lengths_data.reduce(lambda a, b: a+b)
total_length

44